In [26]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [27]:
from conllu import TokenList
import polars
import stanza
from stanza.models.common.doc import Token
from label_legends.preprocess import create_conllu, holdout, load_conllu, load_data, load_train, transform, load_vectorizer, reverse_vocabulary, vocabulary, ids_to_tokens, tokens_to_ids, vectorize_tokens, strip_stopwords
import logging


logging.basicConfig(level=logging.INFO)

In [28]:
load_data().collect()

id,rewire_id,text,annotator,label_sexist,label_category,label_vector,split,tokens
i64,str,str,i64,str,str,str,str,list[str]
0,"""sexism2022_english-0""",""" I wonder what keeps that witc…",17,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train""","[""i"", ""wonder"", … ""😄""]"
1,"""sexism2022_english-0""",""" I wonder what keeps that witc…",2,"""sexist""","""2. derogation""","""2.2 aggressive and emotive att…","""train""","[""i"", ""wonder"", … ""😄""]"
10,"""sexism2022_english-100""","""Good for her! My grandson had …",3,"""not sexist""","""none""","""none""","""train""","[""good"", ""for"", … ""!""]"
100,"""sexism2022_english-10026""","""It is not insulting, it's **ex…",14,"""sexist""","""2. derogation""","""2.1 descriptive attacks""","""test""","[""it"", ""be"", … "".**""]"
1000,"""sexism2022_english-10297""","""The article said Madonna offer…",5,"""sexist""","""2. derogation""","""2.3 dehumanising attacks & ove…","""train""","[""the"", ""article"", … "".""]"
…,…,…,…,…,…,…,…,…
9995,"""sexism2022_english-12996""","""Shudder.. if you had to have s…",17,"""sexist""","""2. derogation""","""2.3 dehumanising attacks & ove…","""test""","[""shudder"", "".."", … "".""]"
9996,"""sexism2022_english-12997""","""You mean one that forces women…",6,"""not sexist""","""none""","""none""","""train""","[""you"", ""mean"", … ""?""]"
9997,"""sexism2022_english-12997""","""You mean one that forces women…",4,"""not sexist""","""none""","""none""","""train""","[""you"", ""mean"", … ""?""]"


In [34]:
val, tra = holdout()
tra = transform(tra)
val = transform(val)

In [37]:
tra

id,text,tokens,token_ids,label
i64,str,list[str],list[i64],str
26016,"""#VoteDemOut seditionist wanna …","[""#"", ""votedemout"", … ""ven""]","[7, 3000, … 3000]","""0"""
41060,"""Cerno hops from wave to wave: …","[""cerno"", ""hop"", … "".""]","[3000, 3000, … 26]","""1"""
35766,"""think about it.. he´s called J…","[""think"", "".."", … ""elect""]","[2688, 27, … 911]","""0"""
23678,"""Hmm, you could rewrite this wi…","[""hmm"", "","", … ""justsaying""]","[1284, 22, … 3000]","""0"""
52396,"""Every girl in a game, or on si…","[""girl"", ""game"", … "".""]","[1164, 1135, … 26]","""0"""
…,…,…,…,…
6785,"""I just think my dick couldn't …","[""just"", ""think"", … ""..😂😂😂""]","[1482, 2688, … 3000]","""0"""
31355,"""no he deadasss hit the nail on…","[""deadass"", ""hit"", … "".""]","[3000, 1282, … 26]","""0"""
49834,"""Aren't these goblins always wh…","[""goblins"", ""whine"", … ""hastagsarestillcool""]","[3000, 2914, … 3000]","""0"""
